In [1]:
# Copyright 2022 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# =======

<img src="https://developer.download.nvidia.com/notebooks/dlsw-notebooks/merlin_transformers4rec_getting-started-session-based-01-etl-with-nvtabular/nvidia_logo.png" style="width: 90px; float: right;">

# ETL with NVTabular

In this notebook we are going to generate synthetic data and then create sequential features with [NVTabular](https://github.com/NVIDIA-Merlin/NVTabular). Such data will be used in the next notebook to train a session-based recommendation model.

NVTabular is a feature engineering and preprocessing library for tabular data designed to quickly and easily manipulate terabyte scale datasets used to train deep learning based recommender systems. It provides a high level abstraction to simplify code and accelerates computation on the GPU using the RAPIDS cuDF library.

### Import required libraries

In [2]:
import os
import glob

import numpy as np
import pandas as pd

import cudf
import cupy as cp
import nvtabular as nvt
from nvtabular.ops import *
from merlin.schema.tags import Tags

### Define Input/Output Path

In [3]:
INPUT_DATA_DIR = os.environ.get("INPUT_DATA_DIR", "/workspace/data/")

## Create a Synthetic Input Data

In [4]:
NUM_ROWS = 100000
long_tailed_item_distribution = np.clip(np.random.lognormal(3., 1., NUM_ROWS).astype(np.int32), 1, 50000)

# generate random item interaction features 
df = pd.DataFrame(np.random.randint(70000, 90000, NUM_ROWS), columns=['session_id'])
df['item_id'] = long_tailed_item_distribution

# generate category mapping for each item-id
df['category'] = pd.cut(df['item_id'], bins=334, labels=np.arange(1, 335)).astype(np.int32)
df['age_days'] = np.random.uniform(0, 1, NUM_ROWS).astype(np.float32)
df['weekday_sin']= np.random.uniform(0, 1, NUM_ROWS).astype(np.float32)

# generate day mapping for each session 
map_day = dict(zip(df.session_id.unique(), np.random.randint(1, 10, size=(df.session_id.nunique()))))
df['day'] =  df.session_id.map(map_day)

Visualize couple of rows of the synthetic dataset:

In [5]:
df.head()

,session_id,item_id,category,age_days,weekday_sin,day
0,88504,26,4,0.342991,0.144433,1
1,85107,13,2,0.156982,0.722122,4
2,89499,37,5,0.389054,0.321258,9
3,88602,20,3,0.258130,0.491159,2
4,84113,7,1,0.519515,0.110561,7


## Feature Engineering with NVTabular

Deep Learning models require dense input features. Categorical features are sparse, and need to be represented by dense embeddings in the model. To allow for that, categorical features first need to be encoded as contiguous integers `(0, ..., |C|)`, where `|C|` is the feature cardinality (number of unique values), so that their embeddings can be efficiently stored in embedding layers.  We will use NVTabular to preprocess the categorical features, so that all categorical columns are encoded as contiguous integers.  Note that in the `Categorify` op, we set `start_index=1`; the reason for that is, we want the encoded null values to start from `1` instead of `0` because we reserve `0` for padding the sequence features.

Here our goal is to create sequential features.  In this cell, we are creating temporal features and grouping them together at the session level, sorting the interactions by time. Note that we also trim each feature sequence in a  session to a certain length. Here, we use the NVTabular library so that we can easily preprocess and create features on GPU with a few lines.

In [6]:
SESSIONS_MAX_LENGTH =20

# Categorify categorical features
categ_feats = ['session_id', 'item_id', 'category'] >> nvt.ops.Categorify(start_index=1)

# Define Groupby Workflow
groupby_feats = categ_feats + ['day', 'age_days', 'weekday_sin']

# Group interaction features by session
groupby_features = groupby_feats >> nvt.ops.Groupby(
    groupby_cols=["session_id"], 
    aggs={
        "item_id": ["list", "count"],
        "category": ["list"],     
        "day": ["first"],
        "age_days": ["list"],
        'weekday_sin': ["list"],
        },
    name_sep="-")

# Select and truncate the sequential features
sequence_features_truncated = (
    groupby_features['category-list']
    >> nvt.ops.ListSlice(-SESSIONS_MAX_LENGTH) 
    >> nvt.ops.ValueCount()
)

sequence_features_truncated_item = (
    groupby_features['item_id-list']
    >> nvt.ops.ListSlice(-SESSIONS_MAX_LENGTH) 
    >> TagAsItemID()
    >> nvt.ops.ValueCount()
)  
sequence_features_truncated_cont = (
    groupby_features['age_days-list', 'weekday_sin-list'] 
    >> nvt.ops.ListSlice(-SESSIONS_MAX_LENGTH) 
    >> nvt.ops.AddMetadata(tags=[Tags.CONTINUOUS])
    >> nvt.ops.ValueCount()
)

# Filter out sessions with length 1 (not valid for next-item prediction training and evaluation)
MINIMUM_SESSION_LENGTH = 2
selected_features = (
    groupby_features['item_id-count', 'day-first', 'session_id'] + 
    sequence_features_truncated_item +
    sequence_features_truncated + 
    sequence_features_truncated_cont
)
    
filtered_sessions = selected_features >> nvt.ops.Filter(f=lambda df: df["item_id-count"] >= MINIMUM_SESSION_LENGTH)

seq_feats_list = filtered_sessions['item_id-list', 'category-list', 'age_days-list', 'weekday_sin-list'] >>  nvt.ops.ValueCount()


workflow = nvt.Workflow(filtered_sessions['session_id', 'day-first', 'item_id-count'] + seq_feats_list)
dataset = nvt.Dataset(df, cpu=False)
# Generate statistics for the features
workflow.fit(dataset)
# Apply the preprocessing and return an NVTabular dataset
sessions_ds = workflow.transform(dataset)
# Convert the NVTabular dataset to a Dask cuDF dataframe (`to_ddf()`) and then to cuDF dataframe (`.compute()`)
sessions_gdf = sessions_ds.to_ddf().compute()

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


In [7]:
sessions_gdf.head(3)

,session_id,day-first,item_id-count,item_id-list,category-list,age_days-list,weekday_sin-list
0,2,9,78,"[19, 15, 26, 6, 24, 44, 19, 29, 14, 42, 33, 11...","[4, 4, 5, 2, 5, 7, 4, 5, 2, 7, 2, 3, 2, 3, 2, ...","[0.0042280755, 0.40522072, 0.42538044, 0.97327...","[0.6292621, 0.1172376, 0.18633945, 0.8232658, ..."
1,3,2,76,"[26, 85, 9, 57, 17, 7, 9, 41, 37, 11, 10, 16, ...","[5, 13, 2, 9, 4, 3, 2, 7, 6, 3, 3, 4, 7, 5, 2,...","[0.07340249, 0.2910817, 0.010784109, 0.8495507...","[0.52355134, 0.83093345, 0.8837344, 0.38942775..."
2,4,6,76,"[13, 13, 50, 64, 105, 16, 78, 17, 19, 34, 8, 1...","[3, 3, 8, 10, 16, 4, 12, 4, 4, 6, 3, 4, 3, 6, ...","[0.29271448, 0.59962034, 0.042938035, 0.730446...","[0.8610789, 0.058191676, 0.806903, 0.79222715,..."


It is possible to save the preprocessing workflow. That is useful to apply the same preprocessing to other data (with the same schema) and also to deploy the session-based recommendation pipeline to Triton Inference Server.

In [8]:
workflow.output_schema

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.start_index,properties.cat_path,properties.domain.min,properties.domain.max,properties.domain.name,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.value_count.min,properties.value_count.max
0,session_id,(Tags.CATEGORICAL),int64,False,False,NaN,0.0,0.0,1.0,.//categories/unique.session_id.parquet,0.0,20001.0,session_id,20002.0,410.0,NaN,NaN
1,day-first,(),int64,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,item_id-count,(Tags.CATEGORICAL),int32,False,False,NaN,0.0,0.0,1.0,.//categories/unique.item_id.parquet,0.0,828.0,item_id,829.0,69.0,NaN,NaN
3,item_id-list,"(Tags.ITEM_ID, Tags.ITEM, Tags.CATEGORICAL, Ta...",int64,True,False,NaN,0.0,0.0,1.0,.//categories/unique.item_id.parquet,0.0,828.0,item_id,829.0,69.0,20.0,20.0
4,category-list,"(Tags.CATEGORICAL, Tags.LIST)",int64,True,False,NaN,0.0,0.0,1.0,.//categories/unique.category.parquet,0.0,172.0,category,173.0,29.0,20.0,20.0
5,age_days-list,"(Tags.CONTINUOUS, Tags.LIST)",float32,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,20.0
6,weekday_sin-list,"(Tags.CONTINUOUS, Tags.LIST)",float32,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,20.0


The following will generate `schema.pbtxt` file in the provided folder.

In [9]:
workflow.fit_transform(dataset).to_parquet(os.path.join(INPUT_DATA_DIR, "processed_nvt"))

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


In [10]:
workflow.save('workflow_etl')

## Export pre-processed data by day

In this example we are going to split the preprocessed parquet files by days, to allow for temporal training and evaluation. There will be a folder for each day and three parquet files within each day folder: `train.parquet`, `validation.parquet` and `test.parquet`.

In [11]:
OUTPUT_DIR = os.environ.get("OUTPUT_DIR",os.path.join(INPUT_DATA_DIR, "sessions_by_day"))
!mkdir -p $OUTPUT_DIR

In [12]:
from transformers4rec.data.preprocessing import save_time_based_splits
save_time_based_splits(data=nvt.Dataset(sessions_gdf),
                       output_dir= OUTPUT_DIR,
                       partition_col='day-first',
                       timestamp_col='session_id', 
                      )

Creating time-based splits: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00,  9.56it/s]


## Checking the preprocessed outputs

In [13]:
TRAIN_PATHS = sorted(glob.glob(os.path.join(OUTPUT_DIR, "1", "train.parquet")))

In [14]:
gdf = cudf.read_parquet(TRAIN_PATHS[0])
gdf

,session_id,item_id-count,item_id-list,category-list,age_days-list,weekday_sin-list
0,15,73,"[7, 15, 7, 18, 49, 106, 6, 2, 9, 9, 10, 31, 2,...","[3, 4, 3, 4, 8, 16, 2, 2, 2, 2, 3, 6, 2, 3, 3,...","[0.83090544, 0.80508804, 0.77800703, 0.1136483...","[0.73860276, 0.71227425, 0.82740945, 0.7208882..."
1,33,72,"[21, 15, 11, 34, 13, 2, 35, 16, 8, 16, 2, 72, ...","[2, 4, 3, 6, 3, 2, 6, 4, 3, 4, 2, 11, 4, 5, 6,...","[0.091410436, 0.39586508, 0.5213629, 0.73786, ...","[0.81503123, 0.8240082, 0.13869828, 0.00287529..."
2,50,71,"[6, 113, 59, 16, 12, 209, 3, 181, 40, 6, 13, 3...","[2, 17, 9, 4, 3, 29, 2, 26, 7, 2, 3, 6, 9, 27,...","[0.0334595, 0.7818315, 0.9511686, 0.48197943, ...","[0.5987661, 0.2299972, 0.57004195, 0.93016136,..."
4,54,71,"[121, 23, 21, 46, 18, 68, 39, 103, 53, 15, 3, ...","[18, 5, 2, 7, 4, 11, 7, 15, 9, 4, 2, 2, 4, 5, ...","[0.24764787, 0.7796014, 0.6935816, 0.37522456,...","[0.7501101, 0.8012419, 0.6419888, 0.18589461, ..."
5,60,70,"[4, 25, 62, 11, 94, 14, 13, 28, 8, 38, 47, 6, ...","[2, 5, 10, 3, 14, 2, 3, 5, 3, 6, 8, 2, 17, 3, ...","[0.42777818, 0.93942195, 0.7336651, 0.6372542,...","[0.6628856, 0.19497228, 0.34514892, 0.7849939,..."
...,...,...,...,...,...,...
2234,19970,31,"[16, 21, 8, 16, 17, 118, 146, 32, 16, 89, 52, ...","[4, 2, 3, 4, 4, 17, 22, 6, 4, 14, 8, 2, 2, 17,...","[0.39309493, 0.80350375, 0.41615465, 0.7130491...","[0.4812473, 0.19307572, 0.31647742, 0.48890728..."
2235,19979,30,"[17, 13, 227, 22, 13, 43, 20, 65, 8, 13, 3, 36...","[4, 3, 33, 4, 3, 7, 4, 10, 3, 3, 2, 6, 2, 2, 7...","[0.80101305, 0.87425673, 0.072974995, 0.940252...","[0.9547868, 0.22636005, 0.14399427, 0.25720155..."
2236,19980,30,"[12, 26, 16, 50, 23, 113, 38, 4, 17, 5, 22, 15...","[3, 5, 4, 8, 5, 17, 6, 2, 4, 3, 4, 4, 3, 3, 3,...","[0.8601423, 0.94999486, 0.19948259, 0.00668230...","[0.53358024, 0.03624035, 0.36104643, 0.6872657..."
2237,19986,29,"[154, 18, 28, 216, 173, 148, 22, 72, 28, 21, 2...","[24, 4, 5, 32, 25, 22, 4, 11, 5, 2, 5, 2, 2, 2...","[0.5032084, 0.22921799, 0.5817064, 0.31098822,...","[0.93975395, 0.9232329, 0.292329, 0.034402855,..."


You have  just created session-level features to train a session-based recommendation model using NVTabular. Now you can move to the the next notebook,`02-session-based-XLNet-with-PyT.ipynb` to train a session-based recommendation model using [XLNet](https://arxiv.org/abs/1906.08237), one of the state-of-the-art NLP model.